<a href="https://colab.research.google.com/github/Juyudang/qsimcirq_tutorial/blob/main/docs/tutorials/qsimcirq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2020 Google

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Get started with qsimcirq

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://quantumai.google/qsim/tutorials/qsimcirq"><img src="https://quantumai.google/site-assets/images/buttons/quantumai_logo_1x.png" />View on QuantumAI</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/quantumlib/qsim/blob/master/docs/tutorials/qsimcirq.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/colab_logo_1x.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/quantumlib/qsim/blob/master/docs/tutorials/qsimcirq.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/github_logo_1x.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/qsim/docs/tutorials/qsimcirq.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/download_icon_1x.png" />Download notebook</a>
  </td>
</table>

The qsim library provides a Python interface to Cirq in the **qsimcirq** PyPI package.

## Setup

Install the Cirq and qsimcirq packages:

In [1]:
try:
    import cirq
except ImportError:
    !pip install cirq --quiet
    import cirq

try:
    import qsimcirq
except ImportError:
    !pip install qsimcirq --quiet
    import qsimcirq

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-events 0.9.0 requires jsonschema[format-nongpl]>=4.18.0, but you have jsonschema 4.17.3 which is incompatible.
jupyterlab 4.1.3 requires httpx>=0.25.0, but you have httpx 0.23.3 which is incompatible.
jupyterlab-server 2.25.3 requires jsonschema>=4.18.0, but you have jsonschema 4.17.3 which is incompatible.
referencing 0.33.0 requires attrs>=22.2.0, but you have attrs 21.4.0 which is incompatible.


Simulating Cirq circuits with qsim is easy: just define the circuit as you normally would, then create a `QSimSimulator` to perform the simulation. This object implements Cirq's [simulator.py](https://github.com/quantumlib/Cirq/blob/master/cirq-core/cirq/sim/simulator.py) interfaces, so you can drop it in anywhere the basic Cirq simulator is used.

## Full state-vector simulation

qsim is optimized for computing the final state vector of a circuit. Try it by running the example below.

In [2]:
# Define qubits and a short circuit.
q0, q1 = cirq.LineQubit.range(2)
circuit = cirq.Circuit(cirq.H(q0), cirq.CX(q0, q1))
print("Circuit:")
print(circuit)
print()

# Simulate the circuit with Cirq and return the full state vector.
print('Cirq results:')
cirq_simulator = cirq.Simulator()
cirq_results = cirq_simulator.simulate(circuit)
print(cirq_results)
print()

# Simulate the circuit with qsim and return the full state vector.
print('qsim results:')
qsim_simulator = qsimcirq.QSimSimulator()
qsim_results = qsim_simulator.simulate(circuit)
print(qsim_results)

Circuit:
0: ───H───@───
          │
1: ───────X───

Cirq results:
measurements: (no measurements)

qubits: (cirq.LineQubit(0), cirq.LineQubit(1))
output vector: 0.707|00⟩ + 0.707|11⟩

phase:
output vector: |⟩

qsim results:
measurements: (no measurements)

qubits: (cirq.LineQubit(0), cirq.LineQubit(1))
output vector: 0.707|00⟩ + 0.707|11⟩


To sample from this state, you can invoke Cirq's `sample_state_vector` method:

In [3]:
samples = cirq.sample_state_vector(
    qsim_results.state_vector(), indices=[0, 1], repetitions=10)
print(samples)

[[1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [1 1]]


## Measurement sampling

qsim also supports sampling from user-defined measurement gates.

> *Note*: Since qsim and Cirq use different random number generators, identical runs on both simulators may give different results, even if they use the same seed.

In [4]:
# Define a circuit with measurements.
q0, q1 = cirq.LineQubit.range(2)
circuit = cirq.Circuit(
    cirq.H(q0), cirq.X(q1), cirq.CX(q0, q1),
    cirq.measure(q0, key='qubit_0'),
    cirq.measure(q1, key='qubit_1'),
)
print("Circuit:")
print(circuit)
print()

# Simulate the circuit with Cirq and return just the measurement values.
print('Cirq results:')
cirq_simulator = cirq.Simulator()
cirq_results = cirq_simulator.run(circuit, repetitions=5)
print(cirq_results)
print()

# Simulate the circuit with qsim and return just the measurement values.
print('qsim results:')
qsim_simulator = qsimcirq.QSimSimulator()
qsim_results = qsim_simulator.run(circuit, repetitions=5)
print(qsim_results)

Circuit:
0: ───H───@───M('qubit_0')───
          │
1: ───X───X───M('qubit_1')───

Cirq results:
qubit_0=11000
qubit_1=00111

qsim results:
qubit_0=00111
qubit_1=11000


The warning above highlights an important distinction between the `simulate` and `run` methods:

* `simulate` only executes the circuit once.
  -  Sampling from the resulting state is fast, but if there are intermediate measurements the final state vector depends on the results of those measurements.
* `run` will execute the circuit once for each repetition requested.
  -  As a result, sampling is much slower, but intermediate measurements are re-sampled for each repetition. If there are no intermediate measurements, `run` redirects to `simulate` for faster execution.

The warning goes away if intermediate measurements are present:

In [5]:
# Define a circuit with intermediate measurements.
q0 = cirq.LineQubit(0)
circuit = cirq.Circuit(
    cirq.X(q0)**0.5, cirq.measure(q0, key='m0'),
    cirq.X(q0)**0.5, cirq.measure(q0, key='m1'),
    cirq.X(q0)**0.5, cirq.measure(q0, key='m2'),
)
print("Circuit:")
print(circuit)
print()

# Simulate the circuit with qsim and return just the measurement values.
print('qsim results:')
qsim_simulator = qsimcirq.QSimSimulator()
qsim_results = qsim_simulator.run(circuit, repetitions=5)
print(qsim_results)

Circuit:
0: ───X^0.5───M('m0')───X^0.5───M('m1')───X^0.5───M('m2')───

qsim results:
m0=01000
m1=10110
m2=11110


## Amplitude evaluation

qsim can also calculate amplitudes for specific output bitstrings.

In [6]:
# Define a simple circuit.
q0, q1 = cirq.LineQubit.range(2)
circuit = cirq.Circuit(cirq.H(q0), cirq.CX(q0, q1))
print("Circuit:")
print(circuit)
print()

# Simulate the circuit with qsim and return the amplitudes for |00) and |01).
print('Cirq results:')
cirq_simulator = cirq.Simulator()
cirq_results = cirq_simulator.compute_amplitudes(
    circuit, bitstrings=[0b00, 0b01])
print(cirq_results)
print()

# Simulate the circuit with qsim and return the amplitudes for |00) and |01).
print('qsim results:')
qsim_simulator = qsimcirq.QSimSimulator()
qsim_results = qsim_simulator.compute_amplitudes(
    circuit, bitstrings=[0b00, 0b01])
print(qsim_results)

Circuit:
0: ───H───@───
          │
1: ───────X───

Cirq results:
[(0.7071067690849304+0j), 0j]

qsim results:
[(0.7071067690849304+0j), 0j]


## Performance benchmark

The code below generates a depth-16 circuit on a 4x5 qubit grid, then runs it against the basic Cirq simulator. For a circuit of this size, the difference in runtime can be significant - try it out!

In [7]:
import time

# Get a rectangular grid of qubits.
qubits = cirq.GridQubit.rect(4, 5)

# Generates a random circuit on the provided qubits.
circuit = cirq.experiments.random_rotations_between_grid_interaction_layers_circuit(
    qubits=qubits, depth=16)

# Simulate the circuit with Cirq and print the runtime.
cirq_simulator = cirq.Simulator()
cirq_start = time.time()
cirq_results = cirq_simulator.simulate(circuit)
cirq_elapsed = time.time() - cirq_start
print(f'Cirq runtime: {cirq_elapsed} seconds.')
print()

# Simulate the circuit with qsim and print the runtime.
qsim_simulator = qsimcirq.QSimSimulator()
qsim_start = time.time()
qsim_results = qsim_simulator.simulate(circuit)
qsim_elapsed = time.time() - qsim_start
print(f'qsim runtime: {qsim_elapsed} seconds.')

Cirq runtime: 4.14240288734436 seconds.

qsim runtime: 0.15659785270690918 seconds.


qsim performance can be tuned further by passing options to the simulator constructor. These options use the same format as the qsim_base binary - a full description can be found in the qsim [usage doc](https://github.com/quantumlib/qsim/blob/master/docs/usage.md). The example below demonstrates enabling multithreading in qsim; for best performance, use the same number of threads as the number of cores (or virtual cores) on your machine.

In [ ]:
# Use eight threads to parallelize simulation.
options = {'t': 8}

qsim_simulator = qsimcirq.QSimSimulator(options)
qsim_start = time.time()
qsim_results = qsim_simulator.simulate(circuit)
qsim_elapsed = time.time() - qsim_start
print(f'qsim runtime: {qsim_elapsed} seconds.')

Another option is to adjust the maximum number of qubits over which to fuse gates. Increasing this value (as demonstrated below) increases arithmetic intensity, which may improve performance with the right environment settings.

In [ ]:
# Increase maximum fused gate size to three qubits.
options = {'f': 3}

qsim_simulator = qsimcirq.QSimSimulator(options)
qsim_start = time.time()
qsim_results = qsim_simulator.simulate(circuit)
qsim_elapsed = time.time() - qsim_start
print(f'qsim runtime: {qsim_elapsed} seconds.')

## Advanced applications: Distributed execution

qsimh (qsim-hybrid) is a second library in the qsim repository that takes a slightly different approach to circuit simulation. When simulating a quantum circuit, it's possible to simplify the execution by decomposing a subset of two-qubit gates into pairs of one-qubit gates with shared indices. This operation is called "slicing" (or "cutting") the gates.

qsimh takes advantage of the "slicing" operation by selecting a set of gates to "slice" and assigning each possible value of the shared indices across a set of executors running in parallel. By adding up the results afterwards, the total state can be recovered.

In [ ]:
# Pick a pair of qubits.
q0 = cirq.GridQubit(0, 0)
q1 = cirq.GridQubit(0, 1)

# Create a circuit that entangles the pair.
circuit = cirq.Circuit(
    cirq.H(q0), cirq.CX(q0, q1), cirq.X(q1)
)
print("Circuit:")
print(circuit)

In order to let qsimh know how we want to split up the circuit, we need to pass it some additional options. More detail on these can be found in the qsim [usage doc](https://github.com/quantumlib/qsim/blob/master/docs/usage.md), but the fundamentals are explained below.

In [ ]:
options = {}

# 'k' indicates the qubits on one side of the cut.
# We'll use qubit 0 for this.
options['k'] = [0]

# 'p' and 'r' control when values are assigned to cut indices.
# There are some intricacies in choosing values for these options,
# but for now we'll set p=1 and r=0.
# This allows us to pre-assign the value of the CX indices
# and distribute its execution to multiple jobs.
options['p'] = 1
options['r'] = 0

# 'w' indicates the value pre-assigned to the cut.
# This should change for each execution.
options['w'] = 0

# Create the qsimh simulator with those options.
qsimh_simulator = qsimcirq.QSimhSimulator(options)
results_0 = qsimh_simulator.compute_amplitudes(
    circuit, bitstrings=[0b00, 0b01, 0b10, 0b11])
print(results_0)

Now to run the other side of the cut...

In [ ]:
options['w'] = 1

qsimh_simulator = qsimcirq.QSimhSimulator(options)
results_1 = qsimh_simulator.compute_amplitudes(
    circuit, bitstrings=[0b00, 0b01, 0b10, 0b11])
print(results_1)

...and add the two together. The results of a normal qsim simulation are shown for comparison.

In [ ]:
results = [r0 + r1 for r0, r1 in zip(results_0, results_1)]
print("qsimh results:")
print(results)

qsim_simulator = qsimcirq.QSimSimulator()
qsim_simulator.compute_amplitudes(circuit, bitstrings=[0b00, 0b01, 0b10, 0b11])
print("qsim results:")
print(results)

The key point to note here is that `results_0` and `results_1` are completely independent - they can be run in parallel on two separate machines, with no communication between the two. Getting the full result requires `2^p` executions, but each individual result is much cheaper to calculate than trying to do the whole circuit at once.

# 작성한 부분

[게이트 변환](https://quantumai.google/cirq/build/gates#measurement_gate)

In [1]:
# 사용 패키지 임포트 부분
import numpy as np      # pi 계산하기위한 numpy
import cirq
import qsimcirq



## Without Noise (ideal condition)

### Fig. 4 (a)

In [2]:
q = cirq.LineQubit(0)

# Create a circuit that rotates the qubit around the X and Y axes simultaneously.
circuit = cirq.Circuit(cirq.ry(0.5 * np.pi)(q))
print("Circuit without measure:")
print(circuit)
print()

# Simulate the circuit with Cirq and return the full state vector.
print('Cirq results:')
cirq_simulator = cirq.Simulator()
cirq_results = cirq_simulator.simulate(circuit)
print(cirq_results)

print()
print("********************")
print()

circuit = cirq.Circuit(cirq.ry(0.5 * np.pi)(q), cirq.measure(q, key='qubit_0'))
print("Circuit with measure:")
print(circuit)
print()

repeted_times = 500
cirq_results = cirq_simulator.run(circuit, repetitions=repeted_times)
print(cirq_results)

counts = cirq_results.histogram(key='qubit_0')
# Calculate probabilities
probabilities = {outcome: count / repeted_times for outcome, count in counts.items()}
print(probabilities)


# # Simulate the circuit with qsim and return the full state vector.
# print('qsim results:')
# qsim_simulator = qsimcirq.QSimSimulator()
# qsim_results = qsim_simulator.simulate(circuit)
# print(qsim_results)

Circuit without measure:
0: ───Ry(0.5π)───

Cirq results:
measurements: (no measurements)

qubits: (cirq.LineQubit(0),)
output vector: 0.707|0⟩ + 0.707|1⟩

phase:
output vector: |⟩

********************

Circuit with measure:
0: ───Ry(0.5π)───M('qubit_0')───

qubit_0=10011011110100100101101100111111101001100100100001001111010010001000110000001100000001000111110010010101011011101001110111101010101100100011001111001111100010011001101110110111101011101110110010010011100001010010111001001010100101011101010110111000110011010011101000011111110101100101111100010110100011010011110011011110110000000111101111110110000101001100101111110011110010111011110101101101011110001001110110100101101111101110110010010111111010001010101000110100001010100110111100001011100101101110
{1: 0.544, 0: 0.456}


### Fig. 4. (b)

In [3]:
q = cirq.LineQubit(0)

# Create a circuit that rotates the qubit around the X and Y axes simultaneously.
circuit = cirq.Circuit( cirq.ry(0.5 * np.pi)(q), cirq.rx(np.pi)(q) )
print("Circuit without measure:")
print(circuit)
print()

# Simulate the circuit with Cirq and return the full state vector.
print('Cirq results:')
cirq_simulator = cirq.Simulator()
cirq_results = cirq_simulator.simulate(circuit)
print(cirq_results)

print()
print("********************")
print()

circuit = cirq.Circuit( cirq.ry(0.5 * np.pi)(q), cirq.rx(np.pi)(q), cirq.measure(q, key='qubit_0'))
print("Circuit with measure:")
print(circuit)
print()

repeted_times = 500
cirq_results = cirq_simulator.run(circuit, repetitions=repeted_times)
print(cirq_results)

counts = cirq_results.histogram(key='qubit_0')
# Calculate probabilities
probabilities = {outcome: count / repeted_times for outcome, count in counts.items()}
print(probabilities)

Circuit without measure:
0: ───Ry(0.5π)───Rx(π)───

Cirq results:
measurements: (no measurements)

qubits: (cirq.LineQubit(0),)
output vector: -0.707j|0⟩ - 0.707j|1⟩

phase:
output vector: |⟩

********************

Circuit with measure:
0: ───Ry(0.5π)───Rx(π)───M('qubit_0')───

qubit_0=10111000011010111001110111000111101011100000101001110011010001010101000011010111001011001001111000111010110000000010001001000011010011101011100101000101010001100001100101000101001010110111101001100001111011011000000110111011111111001111110001110101010010010101111100100011101001111111001000101011101011111100100000000011101111011000000111110010110010110110110000001101101011000001111110111001010100011111100101000101101110011000101001011100101101111100000010000000001110010110001001010001010011100010
{1: 0.498, 0: 0.502}


### Fig. 5 (b) modulation 
1. [Rx(180) X Ry(90)]
    R = Ry(π/2)⋅Rx(π)

In [6]:
q = cirq.LineQubit(0)

custom_matrix = np.array( [ [-1j/(2**0.5), 1j/(2**0.5)], [1/2**0.5, 1/2**0.5] ] )
custom_gate = cirq.MatrixGate(custom_matrix)

# Create a circuit that rotates the qubit around the X and Y axes simultaneously.
circuit = cirq.Circuit(custom_gate(q))
print("Circuit without measure:")
print(circuit)
print()

# Simulate the circuit with Cirq and return the full state vector.
print('Cirq results:')
cirq_simulator = cirq.Simulator()
cirq_results = cirq_simulator.simulate(circuit)
print(cirq_results)

print()
print("********************")
print()

circuit = cirq.Circuit( custom_gate(q), cirq.measure(q, key='qubit_0'))
print("Circuit with measure:")
print(circuit)
print()

repeted_times = 500
cirq_results = cirq_simulator.run(circuit, repetitions=repeted_times)
print(cirq_results)

counts = cirq_results.histogram(key='qubit_0')
# Calculate probabilities
probabilities = {outcome: count / repeted_times for outcome, count in counts.items()}
print(probabilities)

Circuit without measure:
      ┌                           ┐
0: ───│-0.   -0.707j  0.   +0.707j│───
      │ 0.707+0.j     0.707+0.j   │
      └                           ┘

Cirq results:
measurements: (no measurements)

qubits: (cirq.LineQubit(0),)
output vector: -0.707j|0⟩ + 0.707|1⟩

phase:
output vector: |⟩

********************

Circuit with measure:
      ┌                           ┐
0: ───│-0.   -0.707j  0.   +0.707j│───M('qubit_0')───
      │ 0.707+0.j     0.707+0.j   │
      └                           ┘

qubit_0=0011100000001011101100111001101100100110000010100000010010101100000111010111000110011110110000011000001010010110001010111101100111011110010010101011011001110111100010010100001000100111001010110000000110001110001001110000010111110001111111111001010010010011001100110000000100000000100110011010101111001011001010000101011100010010111101001010001111101010100000000001000100110001100111011101011000101010110010010110100011011111001011001001111110110001101000010000111000111001

### Fig. 5 (b) modulation 
2. [Ry(90) X Rx(180)]
    R = Rx(π)⋅Ry(π/2)

In [7]:
q = cirq.LineQubit(0)

custom_matrix = np.array( [ [-1/(2**0.5), 1/(2**0.5)], [-1j/(2**0.5), -1j/(2**0.5)] ] )
custom_gate = cirq.MatrixGate(custom_matrix)

# Create a circuit that rotates the qubit around the X and Y axes simultaneously.
circuit = cirq.Circuit(custom_gate(q))
print("Circuit without measure:")
print(circuit)
print()

# Simulate the circuit with Cirq and return the full state vector.
print('Cirq results:')
cirq_simulator = cirq.Simulator()
cirq_results = cirq_simulator.simulate(circuit)
print(cirq_results)

print()
print("********************")
print()

circuit = cirq.Circuit( custom_gate(q), cirq.measure(q, key='qubit_0'))
print("Circuit with measure:")
print(circuit)
print()

repeted_times = 500
cirq_results = cirq_simulator.run(circuit, repetitions=repeted_times)
print(cirq_results)

counts = cirq_results.histogram(key='qubit_0')
# Calculate probabilities
probabilities = {outcome: count / repeted_times for outcome, count in counts.items()}
print(probabilities)

Circuit without measure:
      ┌                           ┐
0: ───│-0.707+0.j     0.707+0.j   │───
      │-0.   -0.707j -0.   -0.707j│
      └                           ┘

Cirq results:
measurements: (no measurements)

qubits: (cirq.LineQubit(0),)
output vector: -0.707|0⟩ - 0.707j|1⟩

phase:
output vector: |⟩

********************

Circuit with measure:
      ┌                           ┐
0: ───│-0.707+0.j     0.707+0.j   │───M('qubit_0')───
      │-0.   -0.707j -0.   -0.707j│
      └                           ┘

qubit_0=0110111001001101011100000100101110110011100101100101100000010101011100101110001001011111010111101110111110011110101111011110101001110011000010001011110111101000101100110011000010110111111000000000111110011111001011100011010100110000110001000100010011110101010000110100011101001100100101011010001001010001110100101000000111001000100001100011000010001110010101001101000111011111100110100100110100010100010111010001111011011111011011000011101010001110011000101001010011001011

### Fig. 7 Modified Hadamard
1. [Rx(180) X Ry(90)]
    R = Ry(-π/2) ⋅ (2Ry(π/2)⋅Rx(π)

In [8]:
q = cirq.LineQubit(0)

custom_matrix = np.array( [ [-1j/(2**0.5), 1j/(2**0.5)], [1/2**0.5, 1/2**0.5] ] )
custom_gate = cirq.MatrixGate(custom_matrix)

# Create a circuit that rotates the qubit around the X and Y axes simultaneously.
circuit = cirq.Circuit(custom_gate(q), cirq.ry(-0.5 * np.pi)(q))
print("Circuit without measure:")
print(circuit)
print()

# Simulate the circuit with Cirq and return the full state vector.
print('Cirq results:')
cirq_simulator = cirq.Simulator()
cirq_results = cirq_simulator.simulate(circuit)
print(cirq_results)

print()
print("********************")
print()

circuit = cirq.Circuit( custom_gate(q), cirq.ry(-0.5 * np.pi)(q), cirq.measure(q, key='qubit_0'))
print("Circuit with measure:")
print(circuit)
print()

repeted_times = 500
cirq_results = cirq_simulator.run(circuit, repetitions=repeted_times)
print(cirq_results)

counts = cirq_results.histogram(key='qubit_0')
# Calculate probabilities
probabilities = {outcome: count / repeted_times for outcome, count in counts.items()}
print(probabilities)

Circuit without measure:
      ┌                           ┐
0: ───│-0.   -0.707j  0.   +0.707j│───Ry(-0.5π)───
      │ 0.707+0.j     0.707+0.j   │
      └                           ┘

Cirq results:
measurements: (no measurements)

qubits: (cirq.LineQubit(0),)
output vector: (0.5-0.5j)|0⟩ + (0.5+0.5j)|1⟩

phase:
output vector: |⟩

********************

Circuit with measure:
      ┌                           ┐
0: ───│-0.   -0.707j  0.   +0.707j│───Ry(-0.5π)───M('qubit_0')───
      │ 0.707+0.j     0.707+0.j   │
      └                           ┘

qubit_0=11010010001110110010011110010100110000100110101110010110101111111110110001001001100001010011000110001111110010010011101110000111000010100110010011001010100111010101101001001001110110011111010110010000010010111111010010100110010111111010011100010010010111011000101111111001101010100000110100001010111100110110010001111110011101110100001110100001001010101000001010010110111100111100011101001010011011101010101010110010000000101100101110111100

### Fig. 5 (b) modulation 
2. [Ry(90) X Rx(180)]
    R = Ry(-π/2) ⋅ Rx(π)⋅Ry(π/2)

In [9]:
q = cirq.LineQubit(0)

custom_matrix = np.array( [ [-1/(2**0.5), 1/(2**0.5)], [-1j/(2**0.5), -1j/(2**0.5)] ] )
custom_gate = cirq.MatrixGate(custom_matrix)

# Create a circuit that rotates the qubit around the X and Y axes simultaneously.
circuit = cirq.Circuit(custom_gate(q), cirq.ry(-0.5 * np.pi)(q))
print("Circuit without measure:")
print(circuit)
print()

# Simulate the circuit with Cirq and return the full state vector.
print('Cirq results:')
cirq_simulator = cirq.Simulator()
cirq_results = cirq_simulator.simulate(circuit)
print(cirq_results)

print()
print("********************")
print()

circuit = cirq.Circuit( custom_gate(q), cirq.ry(-0.5 * np.pi)(q), cirq.measure(q, key='qubit_0'))
print("Circuit with measure:")
print(circuit)
print()

repeted_times = 500
cirq_results = cirq_simulator.run(circuit, repetitions=repeted_times)
print(cirq_results)

counts = cirq_results.histogram(key='qubit_0')
# Calculate probabilities
probabilities = {outcome: count / repeted_times for outcome, count in counts.items()}
print(probabilities)

Circuit without measure:
      ┌                           ┐
0: ───│-0.707+0.j     0.707+0.j   │───Ry(-0.5π)───
      │-0.   -0.707j -0.   -0.707j│
      └                           ┘

Cirq results:
measurements: (no measurements)

qubits: (cirq.LineQubit(0),)
output vector: (-0.5-0.5j)|0⟩ + (0.5-0.5j)|1⟩

phase:
output vector: |⟩

********************

Circuit with measure:
      ┌                           ┐
0: ───│-0.707+0.j     0.707+0.j   │───Ry(-0.5π)───M('qubit_0')───
      │-0.   -0.707j -0.   -0.707j│
      └                           ┘

qubit_0=1101110011111010110111101011111111111011111101011000111111011101111111110001111001010000101110100100101000110001011110010011000101101100000111101110011110000001111010111101111001110000010000001111010010111010100110000110111010101110011100011100101000101010001001111111110101110100010011101001011001000110001010100001000100100001011001110111010010110011100101110110011000100111001001100001011110000001111101010000011111011111001100000010010

## Noisy Env

In [12]:
# Basic simulation of rotate X_90 and Y_90 (1/2 pi)
bit_flip = cirq.bit_flip(p=0.1)
nqubits = bit_flip.num_qubits()

# q = cirq.LineQubit(0)
q = bit_flip.on_each(cirq.LineQubit.range(0))
# Create a circuit that rotates the qubit around the X and Y axes simultaneously.
circuit = cirq.Circuit(cirq.rx(0.5 * np.pi)(q), cirq.ry(0.5 * np.pi)(q), cirq.measure(q, key='qubit_0'))
print("Circuit:")
print(circuit)
print()

# Simulate the circuit with Cirq and return the full state vector.
print('Cirq results:')
cirq_simulator = cirq.Simulator()
cirq_results = cirq_simulator.simulate(circuit)
print(cirq_results)
print()

# # Simulate the circuit with qsim and return the full state vector.
# print('qsim results:')
# qsim_simulator = qsimcirq.QSimSimulator()
# qsim_results = qsim_simulator.simulate(circuit)
# print(qsim_results)

AttributeError: 'list' object has no attribute 'dimension'

In [ ]:
, cirq.measure(q, key='qubit_0')